# Deploy Pretrained GPT2 using Sagemaker

In [5]:
import sagemaker

sagemaker_session = sagemaker.Session()

#Specify the data bucket, we do not need it in this case
#bucket = sagemaker_session.default_bucket()
#prefix = "sagemaker/test-gpt2"

role = sagemaker.get_execution_role()

In [6]:
#!pip install sagemaker --upgrade

# deploy the model using `sagemaker.pytorch.PyTorch`

Check out the src folder to see the code for the model (`save_sagemaker.py`) and an additional `requirements.txt` that specify the extra libraries we need.

In [7]:
from sagemaker.pytorch import PyTorch
from sagemaker.huggingface import HuggingFace

estimator = PyTorch(
    entry_point="save_sagemaker.py",
    source_dir="./src/code",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=1,
    instance_type="ml.m4.xlarge"
)
# estimator = HuggingFace(
#         entry_point='train.py',
#         source_dir='./scripts',
#         instance_type='ml.p3.2xlarge',
#         instance_count=1,
#         role=role,
#         transformers_version='4.26',
#         pytorch_version='1.13',
#         py_version='py39'
# )

# Download the pretrained model to a S3 bucket
No actual training is involved in this step

In [25]:
estimator.fit()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-03-27-02-20-37-037


2023-03-27 02:20:38 Starting - Starting the training job...
2023-03-27 02:21:03 Starting - Preparing the instances for training......
2023-03-27 02:22:09 Downloading - Downloading input data...
2023-03-27 02:22:34 Training - Downloading the training image............
2023-03-27 02:24:14 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-03-26 22:24:29,875 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-03-26 22:24:29,877 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-03-26 22:24:29,879 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-26 22:24:29,891 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-03-26 22:24:29,894 sagemaker_pytorch_container.training INFO     Invoking user t

# Load model directly from S3 bucket

In [14]:
from sagemaker.pipeline import PipelineModel
from sagemaker.model import Model
from sagemaker.pytorch.model import PyTorchModel

s3_model_bucket = 'gpt2-sagemaker'
s3_model_key_prefix = 'model'
data = 's3://{}/{}/{}'.format(s3_model_bucket, s3_model_key_prefix, 'model.tar.gz')


estimator = PyTorchModel(model_data=data, role=role,
                             entry_point='save_sagemaker.py',
                             py_version="py38",
                             framework_version="1.11.0")

# Deploying the model

In [42]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

INFO:sagemaker:Creating model with name: pytorch-training-2023-03-27-02-59-27-087
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-03-27-02-59-27-087
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-03-27-02-59-27-087


-----!

In [33]:
import torch
@torch.no_grad()
def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
    """
    Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
    the sequence max_new_tokens times, feeding the predictions back into the model each time.
    Most likely you'll want to make sure to be in model.eval() mode of operation for this.
    """
    for _ in range(max_new_tokens):
        # if the sequence context is growing too long we must crop it at block_size
        idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
        # forward the model to get the logits for the index in the sequence
        logits, _ = self(idx_cond)
        # pluck the logits at the final step and scale by desired temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop the logits to only the top k options
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
        # apply softmax to convert logits to (normalized) probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sampled index to the running sequence and continue
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
# Set the sequence length
seq_length = 1

# Create a random tensor of shape (batch_size, seq_length) with values between 0 and 100
batch_size = 1
input_tensor = torch.randint(0, 100, (batch_size, seq_length))

# Create the attention mask
attention_mask = torch.ones(batch_size, seq_length).to(torch.int)

# Create the token type IDs
token_type_ids = torch.zeros(batch_size, seq_length).to(torch.int)

# Create the position IDs
position_ids = torch.arange(seq_length, dtype=torch.int).expand(batch_size, seq_length)

# Create the input tensor for the transformer model
input_ids = input_tensor.to(torch.int64)


In [44]:
input_tensor

tensor([[27]])

In [45]:
predictor.predict(input_ids)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-training-2023-03-27-02-59-27-087 in account 414692569093 for more information.

In [46]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)

INFO:sagemaker:Deleting endpoint with name: pytorch-training-2023-03-27-02-59-27-087
